In [0]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import io
from skimage.feature import hog

In [2]:
from keras.datasets import mnist
(train_image,train_labels), (test_image,test_labels)=mnist.load_data()

Using TensorFlow backend.


In [3]:
X=train_image.reshape((60000, 28*28))
y = train_labels

# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
onehotencoder = OneHotEncoder(categorical_features = [0])
Y = onehotencoder.fit_transform(y.reshape(-1, 1)).toarray()

# To remember train/test split for other notebook
X_train=X
y_train=Y.astype("int")

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)

X_train = X_train.reshape(-1, 28, 28, 1).astype("float32")

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype uint8 w

In [4]:
# Extract the hog features
list_hog_fd = []
for feature in X_train:
    fd = hog(feature.reshape((28, 28)), orientations=9, pixels_per_cell=(7, 7), cells_per_block=(1, 1), visualise=False)
    list_hog_fd.append(fd)
hog_features = np.array(list_hog_fd, 'float64')
hog_features.shape

/usr/local/lib/python3.6/dist-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


(60000, 144)

In [5]:
# # Building the ANN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense,BatchNormalization,Dropout
from keras.callbacks import LearningRateScheduler

classifier = Sequential()

# classifier.add(Flatten())
classifier.add(Dense(1024, activation='relu', input_dim = 144))
classifier.add(Dropout(0.25))
classifier.add(Dense(512, activation='relu'))
classifier.add(Dropout(0.25))
classifier.add(Dense(256, activation='relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(10, activation='softmax'))


classifier.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ["accuracy"])

# Fitting the ANN to the Training set
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)
hist=classifier.fit(hog_features, y_train, batch_size = 100, epochs = 40,callbacks=[annealer])


Epoch 1/40
60000/60000 [==============================] - 6s 98us/step - loss: 0.0569 - acc: 0.9799
Epoch 2/40
60000/60000 [==============================] - 5s 82us/step - loss: 0.0292 - acc: 0.9900
Epoch 3/40
60000/60000 [==============================] - 5s 83us/step - loss: 0.0236 - acc: 0.9919
Epoch 4/40
60000/60000 [==============================] - 5s 82us/step - loss: 0.0212 - acc: 0.9928
Epoch 5/40
60000/60000 [==============================] - 5s 82us/step - loss: 0.0183 - acc: 0.9938
Epoch 6/40
60000/60000 [==============================] - 5s 82us/step - loss: 0.0160 - acc: 0.9944
Epoch 7/40
60000/60000 [==============================] - 5s 82us/step - loss: 0.0144 - acc: 0.9952
Epoch 8/40
60000/60000 [==============================] - 5s 83us/step - loss: 0.0128 - acc: 0.9955
Epoch 9/40
60000/60000 [==============================] - 5s 82us/step - loss: 0.0115 - acc: 0.9961
Epoch 10/40
60000/60000 [==============================] - 5s 84us/step - loss: 0.0105 - acc: 0.9964

In [8]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              148480    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2570      
Total para

In [6]:
# save the model to disk
from sklearn.externals import joblib
filename = 'MNIST_HOG_model.sav'
joblib.dump(classifier, filename)

['MNIST_HOG_model.sav']

In [0]:
from google.colab import files
files.download('MNIST_HOG_model.sav')